<a href="https://colab.research.google.com/github/Mahjabeenqamar1/Real_Fake-FACE-DETECTION/blob/main/Real_Time_Face_Mask_Detector_UET_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

 Build the neural network:



In [ ]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

Image Data Generation/Augmentation

In [ ]:
TRAINING_DIR = "/content/drive/MyDrive/Data_set/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "/content/drive/MyDrive/Data_set/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


Initialize a callback checkpoint to keep saving best model after each epoch while training:

In [ ]:

checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

 Train the model:

In [ ]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
132/132 [==============================] - 922s 7s/step - loss: 0.6486 - acc: 0.6487 - val_loss: 0.2510 - val_acc: 0.9021
INFO:tensorflow:Assets written to: model2-001.model/assets
Epoch 2/10
132/132 [==============================] - 20s 153ms/step - loss: 0.4254 - acc: 0.8411 - val_loss: 0.1524 - val_acc: 0.9536
INFO:tensorflow:Assets written to: model2-002.model/assets
Epoch 3/10
132/132 [==============================] - 20s 152ms/step - loss: 0.3034 - acc: 0.8783 - val_loss: 0.1389 - val_acc: 0.9691
INFO:tensorflow:Assets written to: model2-003.model/assets
Epoch 4/10
132/132 [==============================] - 20s 152ms/step - loss: 0.2725 - acc: 0.9065 - val_loss: 0.0827 - val_acc: 0.9639
INFO:tensorflow:Assets written to: model2-004.model/assets
Epoch 5/10
132/132 [==============================] - 20s 152ms/step - loss: 0.2547 - acc: 0.8981 - val_loss: 0.2606 - val_acc: 0.8814
Epoch 6/10
132/132 [==============================] - 20s 150ms/step - loss: 0.2350 - acc: 

Now we will test the results of face mask detector model using OpenCV.

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("/content/model.h5")

results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture('') 


haarcascade = cv2.CascadeClassifier('/content/drive/MyDrive/New folder/haarcascade_frontalface_default.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()

OSError: ignored